# Módulo 7: Serotipado *Streptococcus pneumoniae* y *Streptococcus agalactiae* 

## Descripción general

Este módulo será desarrollado en dos partes:

1. Parte 1: Prediciendo serotipos de *S. pneumoniae*

2. Parte 2: Prediciendo serotipos de *S. agalactiae*

### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

In [ ]:
# Instalar seroBA
!conda install -c bioconda seroba

In [ ]:
# Instalar seroBA
!conda install bioconda::srst2

In [ ]:
# Compruebe si se seroBA se instaló correctamente
!seroba --version

### Descargar datos

In [ ]:
!wget

## Parte 1: Prediciendo serotipos de *S. pneumoniae*

Hasta la fecha, hay más de 100 serotipos conocidos descritos para *S. pneumoniae* según las diferentes propiedades bioquímicas y antigénicas de la cápsula. Hay una serie de métodos in-silico para detectar el locus cps, que luego se pueden utilizar para predecir serotipos a partir de datos WGS. La identificación precisa de los serotipos neumocócicos es importante para rastrear la distribución y evolución de los serotipos luego de la introducción de vacunas efectivas.

Se desarrolló [SeroBA](https://github.com/sanger-pathogens/seroba?tab=readme-ov-file#installation) que hace un uso eficiente de los recursos computacionales además de detectar con precisión el locus cps en baja cobertura, y predice serotipos a partir de datos WGS utilizando una base de datos adaptada de PneumoCaT. SeroBA puede predecir serotipos mediante la identificación del locus cps, directamente a partir de datos de lectura de secuenciación del genoma completo sin procesar con un 98 % de concordancia utilizando un método basado en k-mer, puede procesar 10 000 muestras en poco más de 1 día utilizando un servidor estándar y puede llamar serotipos en una cobertura tan baja como 10x. SeroBA está implementado en Python3 y está disponible gratuitamente bajo una licencia GPLv3 de código abierto.

*Lectura adicional*: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6113868/

Explore el uso de serobar ejecutando ``seroba -h``

In [ ]:
# Ejecutar seroBA
!seroba -h

SeroBA requiere solo tres archivos de entrada:

1. Base de datos con kmc (utilidad diseñada para contar k-mers) y ariba (identificación de resistencia antimicrobiana por ensamblaje)
2. Archivos de secuencias forward y reverse en fastq
3. Prefijo de salida


Descargargue la base de datos de [PneumoCaT](https://github.com/ukhsa-collaboration/PneumoCaT) usando el comando:

In [ ]:
# Descargar la base de datos
!seroba getPneumocat PneumoCaT_dir

Este comando descarga PneumoCat y a partir de este crea un archivo de metadatos con formato tsvl. Sin embargo, para este módulo usaremos seroba_k71_14082017

### Paso 1: Para predecir el serotipo de una sola cepa (17150_4#79), usaremos el comando:

In [ ]:
# Correr seroBA
!seroba runSerotyping seroba_k71_14082017 17150_4#79_1.fastq.gz 17150_4#79_2.fastq.gz 17150_4#79_output

Una explicación de este comando es la siguiente:

`seroba` es la herramienta/programa

`runSerotyping` especifica que el programa realizará la serotipificación

`seroba_k71_14082017` especifica dónde está el directorio de seroba

`17150_4#79_1.fastq.gz` y `17150_4#79_2.fastq.gz` son los archivos fastq de avance y retroceso

`17150_4#79_output` especifica el prefijo de salida

En la carpeta de salida, encontrará un **pred.tsv** que incluye su serotipo predicho.

### Paso 2: Para predecir el serotipo de múltiples cepas usaremos el comando:

In [ ]:
# Mover archivos
for x in *1.fastq.gz; do mkdir ${x%%_1.fastq.gz} ; mv $x ${x%%_1.fastq.gz}; mv ${x%%1.fastq.gz}2.fastq.gz ${x%%_1.fastq.gz}; done

Una explicación de este comando es la siguiente:

`for x en 1.fastq.gz;` Esto inicia un ciclo for que itera sobre todos los archivos en el directorio actual que terminan con "1.fastq.gz".

`do` Esto inicia el bloque de código que se ejecutará para cada archivo.

`mkdir ${x%%_1.fastq.gz}` Esto crea un nuevo directorio con el mismo nombre que el archivo, pero con el sufijo "_1.fastq.gz" eliminado.

`mv $x ${x%%_1.fastq.gz}` Esto mueve el archivo original al nuevo directorio creado en el paso anterior.

`mv ${x%%1.fastq.gz}2.fastq.gz ${x%%_1.fastq.gz}` Esto mueve un segundo archivo que tiene el mismo prefijo que el primer archivo, pero con un "2.fastq sufijo .gz", en el nuevo directorio creado en el primer paso.

`done` Esto finaliza el bucle for.

En general, este script está diseñado para tomar datos de secuenciación pareados que se almacenan en dos archivos separados con nombres que terminan en "_1.fastq.gz" y "_2.fastq.gz", y organizarlos en directorios según el prefijo de los nombres de archivo.

*Lectura adicional sobre los comandos de bucle*: https://www.gnu.org/software/bash/manual/bash.html#Looping-Constructs

Luego ejecutaremos seroBA usando el comando:

In [ ]:
# Correr seroBA en un loop
for x in *#* ; do seroba runSerotyping seroba_k71_14082017 $x/${x}_1.fastq.gz $x/${x}_2.fastq.gz $x"_output"; done

Una explicación de este comando es la siguiente:

`for x in #;` Esto inicia un ciclo for que itera sobre todos los archivos en el directorio actual que contienen el carácter "#" en su nombre.

`do` Esto inicia el bloque de código que se ejecutará para cada archivo.

`seroba runSerotyping seroba_k71_14082017` Esto ejecuta el comando "runSerotyping" dentro del contenedor Docker, usando la base de datos "seroba_k71_14082017".

`$x/${x}_1.fastq.gz $x/${x}_2.fastq.gz` Estos son los archivos de entrada para el comando, ubicados en un directorio con el nombre del archivo (sin el carácter "#") y con "_1.fastq.gz" o "_2.fastq.gz" añadido al nombre.

`$x"_output"` Este es el directorio de salida para el comando, también nombrado después del archivo (con "_output" añadido al nombre).

`;` Esto separa el comando Docker del final del bucle for.

`done` Esto finaliza el bucle for.

En general, este script está diseñado para ejecutar un análisis de serotipado en datos de secuenciación de extremos pareados que se almacenan en dos archivos separados con nombres que contienen el carácter ``#``. La salida resultante se almacenará en directorios con nombres que se basan en los nombres de los archivos de entrada

### Paso 3: compilaremos los resultados de las ejecuciones anteriores con el comando:

Este comando combinará las salidas de seroba en un archivo tsv.

In [ ]:
# Resumen de los resultados
!seroba summary ./

___

## Parte 2: Prediciendo serotipos de *S. agalactiae*

*Streptococcus agalactiae* (*Streptococcus* del grupo B o GBS) se divide actualmente en diez serotipos según los antígenos capsulares específicos del tipo y se denominan Ia, Ib, II, III, IV, V, VI, VII, VIII y IX.

Group B Streptococcus Serotyping by Genome Sequencing repository contiene un archivo de referencia seleccionado que se puede utilizar para serotipificar *Streptococcus agalactiae* *in silico* con datos de secuenciación del genoma completo. El archivo de referencia (GBS-SBG.fasta) está diseñado para que se pueda utilizar tanto para el mapeo de lectura corta como para las estrategias basadas en ensamblaje.

*Lectura adicional*: https://github.com/swainechen/GBS-SBG



### Paso 1: para predecir el serotipo de una sola cepa (20280_5#33), usaremos el comando:

In [ ]:
# Correr srst2
!srst2 --input_pe 20280_5#33_1.fastq.gz 20280_5#33_2.fastq.gz --output 20280_5#33_test --log --gene_db GBS-SBG.fasta

Una explicación de este comando es la siguiente:

`srst2` es la herramienta

`--input_pe` especifica que el archivo de entrada son lecturas finales emparejadas que son 20280_5#33_1.fastq.gz 20280_5#33_2.fastq.gz

`--output `especifica el archivo de salida 20280_5#33_test

`--log` active el registro en el archivo, en lugar de la salida estándar

`--gene_db` especifica la base de datos GBS-SBG.fasta

Ejecute el comando `ls -lh` para verificar el contenido de la carpeta.

In [ ]:
# Listar archivos
!ls -lh

Obtendrás esta salida


El archivo de salida de la ejecución anterior es "20280_5#33_test__genes__GBS-SBG__results.txt".

Entonces, ejecuta `cat` "20280_5#33_test__genes__GBS-SBG__results.txt" para ver el contenido de este archivo

In [ ]:
# Ver el contenido de un archivo
!cat 20280_5#33_test__fullgenes__GBS-SBG__results.txt

### Paso 2: Para ejecutar SRST2 en múltiples cepas, ejecute el comando:

In [ ]:
# Correr srst2 en un loop
srst2 --input_pe *.fastq.gz --output s.agalactiae --log --gene_db GBS-SBG.fasta

`--input_pe .fastq.gz` especifica que el archivo de entrada son varios archivos fastq.gz comprimidos.

## BONO!

Si está trabajando con BASH en su computadora o en un HPC y tiene demasiados archivos, puede optimizar los comandos, los bucles (for loops) son muy útiles para grandes conjuntos de datos.

Aquí hay una manera de hacerlo.

Cree un nuevo script bash usando nano llamado `serotype.sh`

In [ ]:
#!/bin/bash
#Nombre de archivo: Serotype.sh
#Este script para serotipificar S.pneumoniae a partir de fastq.gz recortados

function docker_run() { docker run --rm=True -u $(id -u):$(id -g) -v $(pwd):/data "$@" ;}
wordir=/home/bioinfo-sanger/Group_work/species/s.pneumo/results_fastqc_gz/trimmed_results/results_trimmed_gz
cd $wordir

for i in $(ls *_1.trimmed.fastq.gz); do

NAME=$(basename $i _1.trimmed.fastq.gz)
echo "$NAME"
j="${NAME}_1.trimmed.fastq.gz"
echo "$j"
k="${NAME}_2.trimmed.fastq.gz"
echo "$k"

docker_run staphb/seroba seroba runSerotyping seroba_k71_14082017 ./$j ./$k ${NAME}_serotype_output;

done

Movemos todas las salidas a una nueva carpeta  `mv *_serotype_output serotype_results`, luego compilamos todos los datos dentro del nuevo directorio con el comando `seroba summary ./`  debe obtener un archivo tsv.

Cree un nuevo script bash usando nano llamado `serotype_2.sh`

In [ ]:
#!/bin/bash
#Este script para serotipificar múltiples lecturas genómicas de S. agalactaeae

function docker_run() { docker run --rm=True -u $(id -u):$(id -g) -v $(pwd):/data "$@" ;}
wordir=/home/bioinfo-sanger/Data/Group_work/s.pneumo/s.agalactiae/lanes_2.txt/
cd $wordir
mkdir -p serotyping_output

for i in $(ls *_1.fastq.gz); do

NAME=$(basename $i _1.fastq.gz)
echo "$NAME"
j="${NAME}_1.fastq.gz"
echo "$j"
k="${NAME}_2.fastq.gz"
echo "$k"

docker_run staphb/srst2 srst2 --input_pe $j $k --output ./serotyping_out/${NAME}_output --log --gene_db analysis/clean_data/GBS-SBG.fasta;

done